In [1]:
# Import Dependencies
import re
import xml.etree.ElementTree as ET
import xmlschema
import pandas as pd
import yaml
from collections import OrderedDict
from ruamel.yaml import YAML

In [2]:
# File paths
yaml_input_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/output_files/L183_S4_Potschke_2003.yaml' 
xml_output_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/input_files/L183_S4_Potschke_2003_convert.xml'

In [8]:
# File paths
#yaml_input_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/output_files/L175_S5_OReilly_2015.yaml'  
#xml_output_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/input_files/L175_S5_OReilly_2015_convert.xml'  

In [13]:

def dict_to_xml(tag, d):
    """
    Converts a dictionary to an XML element.
    
    Args:
    - tag (str): The tag name for the XML root element.
    - d (dict): The dictionary to convert.
    
    Returns:
    - xml.etree.ElementTree.Element: The XML element.
    """
    # Check if the tag name is valid
    if not tag.isidentifier():
        raise ValueError(f"Invalid XML element name: '{tag}'")
    
    elem = ET.Element(tag)
    
    if isinstance(d, dict):
        # Process attributes and text first
        if "@attributes" in d:
            elem.attrib.update(d["@attributes"])
        if "@text" in d:
            elem.text = d["@text"]
        
        # Process other elements
        for key, val in d.items():
            if key in ["@attributes", "@text"]:
                continue
            if not key.isidentifier():
                raise ValueError(f"Invalid XML element name: '{key}'")
            if isinstance(val, dict):
                child = dict_to_xml(key, val)
                elem.append(child)
            elif isinstance(val, list):
                for sub_dict in val:
                    child = dict_to_xml(key, sub_dict)
                    elem.append(child)
            else:
                child = ET.SubElement(elem, key)
                child.text = str(val)
    elif isinstance(d, list):
        for sub_dict in d:
            child = dict_to_xml(tag, sub_dict)
            elem.append(child)
    else:
        elem.text = str(d)
    
    return elem

def write_yaml_to_xml(yaml_file_path, xml_output_path):
    """
    Converts a YAML file to an XML file.
    
    Args:
    - yaml_file_path (str): Path to the input YAML file.
    - xml_output_path (str): Path to the output XML file.
    """
    yaml = YAML()
    with open(yaml_file_path, 'r', encoding='utf-8') as file:
        yaml_data = yaml.load(file)
    
    if isinstance(yaml_data, dict):
        root_tag, root_content = next(iter(yaml_data.items()))
    else:
        raise ValueError("The YAML root should be a dictionary with one root element.")
    
    xml_element = dict_to_xml(root_tag, root_content)
    
    tree = ET.ElementTree(xml_element)
    tree.write(xml_output_path, encoding='utf-8', xml_declaration=True)
    
    print(f'XML file has been generated and saved as {xml_output_path}')

# Convert YAML to XML
write_yaml_to_xml(yaml_input_path, xml_output_path)

XML file has been generated and saved as C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/input_files/L175_S5_OReilly_2015_convert.xml
